## Class Data Construction


#### Table of Contents

* [Overview](#Overview)
* [Setup](#Setup)
* [GDP](#GDP)
* [Unemployment](#Unemployment)
* [BDS](#BDS)
* [CDC](#CDC)
* [Social Explorer](#Social-Explorer)
* [Saving](#Saving)

# Overview

This notebook constructs the data set used for in-class examples and for the workshops.
The data is at the county-year level from 2002-2018.
We will be constructing three different labels that will demonstrate different techniques.

* Labels
    1. Continuous - %$\Delta$rGDP
    2. Binary - $I\{net job creation \geq 0 \}$
    3. Multiclass - County unemployment rate $\in \{\ll U\_rate_{national}, \sim U\_rate_{national}, \gg  U\_rate_{national}\}$
* Feautres
    1. Population
    2. Population density
    3. Percent black
    4. Percent hispanic
    5. Establishment entry rate
    6. Establishment exit rate
    7. Employment per establishments
    8. Quasi LF participation rate
    9. Year Fixed Effects


The data are sourced from five government databases:

1. [BEA](https://apps.bea.gov/regional/downloadzip.cfm) - CAGDP9
2. [BLS Local Area Unemployment Statistics (LAUS)](https://www.bls.gov/lau/tables.htm)  - County Data
3. [Census Business Dynamic Statistics (BDS)](https://www.census.gov/data/tables/time-series/econ/bds/bds-tables.html) - One-Way Tables: County
4. [CDC WONDER](https://wonder.cdc.gov/Bridged-Race-v2019.HTML) - Demographics
5. [Social Explorer](http://www.library.illinois.edu/proxy/go.php?url=http://www.socialexplorer.com/ezproxy) - County land area


***

# Setup

[top](#Overview)

In [1]:
# A cell dedicated to preliminaries
import pandas as pd
import numpy as np
import os

os.chdir('C:/Users/johnj/Documents/Data/aml in econ 02 spring 2021/class data')
print(os.listdir())

['area codebook.txt', 'area.csv', 'bds2018_cty.csv', 'CAGDP9__ALL_AREAS_2001_2019.csv', 'class_data.csv', 'class_data.pkl', 'demographics', 'unemployment']


***
# GDP 

[top](#Overview)

In [2]:
gdp = pd.read_csv('CAGDP9__ALL_AREAS_2001_2019.csv', encoding = 'ANSI', quotechar = '"')
gdp

C:\Users\johnj\anaconda3\envs\classprep\lib\site-packages\IPython\core\interactiveshell.py:3155: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,GeoFIPS,GeoName,Region,TableName,LineCode,IndustryClassification,Description,Unit,2001,2002,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,"""00000""",United States,,CAGDP9,1.0,...,All industry total,Thousands of chained 2012 dollars,13262079000,13493064000,...,15598753000,15840664000,16197007000,16495369000,16912038000,17432170000,17730509000,18144105000,18687786000,19091662000
1,"""00000""",United States,,CAGDP9,2.0,...,Private industries,Thousands of chained 2012 dollars,11465144000,11669109000,...,13467315000,13697390000,14037519000,14302267000,14715952000,15232954000,15511037000,15902123000,16415475000,16804174000
2,"""00000""",United States,,CAGDP9,3.0,11,"Agriculture, forestry, fishing and hunting",Thousands of chained 2012 dollars,156155000,161617000,...,193856000,186395000,179573000,209388000,211758000,227217000,238834000,233675000,243393000,243607000
3,"""00000""",United States,,CAGDP9,6.0,21,"Mining, quarrying, and oil and gas extraction",Thousands of chained 2012 dollars,273325000,280555000,...,309259000,320758000,358798000,372927000,413809000,451834000,427127000,430992000,452225000,504316000
4,"""00000""",United States,,CAGDP9,10.0,22,Utilities,Thousands of chained 2012 dollars,215356000,222899000,...,265789000,275960000,279651000,276620000,265954000,266247000,279573000,280621000,282101000,285691000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107983,"""98000""",Far West,8.0,CAGDP9,92.0,...,Private services-providing industries 3/,Thousands of chained 2012 dollars,1581444302,1636833108,...,1954318205,2018644775,2075109267,2141762495,2232779955,2354663121,2442094736,2567986277,2671918501,2779706298
107984,Note: See the included footnote file.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
107985,CAGDP9: Real GDP by county and metropolitan ar...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
107986,"Last updated: December 9, 2020 -- new statisti...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
gdp.dtypes
# Lots of non numbers

GeoFIPS                    object
GeoName                    object
Region                     object
TableName                  object
LineCode                  float64
IndustryClassification     object
Description                object
Unit                       object
2001                       object
2002                       object
2003                       object
2004                       object
2005                       object
2006                       object
2007                       object
2008                       object
2009                       object
2010                       object
2011                       object
2012                       object
2013                       object
2014                       object
2015                       object
2016                       object
2017                       object
2018                       object
2019                       object
dtype: object

In [4]:
print(gdp['2001'].drop_duplicates().sort_values().head(10), '\n')
print(gdp['2001'].drop_duplicates().sort_values(ascending = False).head(10))

75          (D)
2686       (NA)
241           0
9565          1
31284        10
15357       100
9714       1000
58791     10000
61409    100003
69544    100005
Name: 2001, dtype: object 

48312       99996
66557        9999
100165    9997966
20448       99975
981         99974
40743        9997
22178       99969
49787        9996
95466      999581
49583       99956
Name: 2001, dtype: object


In [5]:
# dealing with and removing NAs
gdp.replace('(NA)', np.nan, inplace = True)
gdp.replace('(D)', np.nan, inplace = True)
gdp.dropna(inplace = True)

# Filter all industry total
gdp = gdp[gdp['LineCode'] == 1].reset_index(drop = True)
    
#######################
## Cleaning BEA Data ##
#######################
# Dealing with BEA... Why do you have '_"12345"'?!?!?!?
gdp['fips'] = gdp['GeoFIPS'].str[-6:-1]
# Omitting non-counties
gdp = gdp[gdp['fips'].str[2:5] != '000']
# Changing fips to a numberic
gdp['fips'] = pd.to_numeric(gdp.fips)
# Checking the dimensions
print(gdp.shape)

#############
## Pivoting
# Adjusting column names to make the pivot easier
old = [str(x) for x in range(2001, 2020)]
new = ['gdp' + str(x) for x in range(2001, 2020)]
gdp.rename(columns = dict(zip(old, new)), inplace = True)

(3107, 28)


In [6]:
df = pd.wide_to_long(gdp, stubnames = ['gdp'], i = 'fips', j = 'year')
##############
## NEW LINE ##
##############
df.set_index('GeoName', append = True, inplace = True)
###################
## Modified Line ##
###################
df = df[['gdp']]
#########################
print(df.gdp.dtype)

# An alternative way to cast 
df['gdp'] = df.gdp.astype(int)

# Creating %\Delta rGDP
df['gdp_old'] = df['gdp'].groupby('fips').shift(1)
df['pct_d_rgdp'] = (df['gdp'] - df['gdp_old'])/df['gdp_old']*100
df = df[['pct_d_rgdp']].dropna()
df.head()

object


,,,pct_d_rgdp
fips,year,GeoName,
1001,2002,"Autauga, AL",3.202147
1003,2002,"Baldwin, AL",5.021265
1005,2002,"Barbour, AL",-0.275078
1007,2002,"Bibb, AL",2.767181
1009,2002,"Blount, AL",0.745068


***
# Unemployment

[top](#Overview)

In [7]:
files = os.listdir('unemployment')
files

['laucnty02.xlsx',
 'laucnty03.xlsx',
 'laucnty04.xlsx',
 'laucnty05.xlsx',
 'laucnty06.xlsx',
 'laucnty07.xlsx',
 'laucnty08.xlsx',
 'laucnty09.xlsx',
 'laucnty10.xlsx',
 'laucnty11.xlsx',
 'laucnty12.xlsx',
 'laucnty13.xlsx',
 'laucnty14.xlsx',
 'laucnty15.xlsx',
 'laucnty16.xlsx',
 'laucnty17.xlsx',
 'laucnty18.xlsx']

In [8]:
# There are many files that we need to load in
# Thank goodness for for loops!
urate = pd.read_excel('unemployment/' + files[0], engine = 'openpyxl',
             skiprows = 5, skipfooter = 3,
             names = ['lau_code', 'fips_state', 'fips_county', 'name', 'year', 'na',
                      'lf', 'emp', 'unemp', 'urate'])

# Trimming Puerto Rico
urate = urate[urate['fips_state'] <= 56]
# Creating FIPS variable
urate['fips'] = urate.fips_state*1000 + urate['fips_county']

# Creating label
urate['urate_nat'] = sum(urate['unemp'])/sum(urate['lf'])*100
urate['urate_diff'] = urate.urate - urate.urate_nat
print(urate.urate_diff.describe())
urate['urate_bin'] = np.where(urate['urate_diff'] < -0.5,
                              'lower',
                              np.where(urate['urate_diff'] <= 0.5,
                                       'similar',
                                       'higher'))
# What percent in which bin?
print(urate.urate_bin.value_counts()/urate.shape[0]*100)

urate = urate[['fips', 'year', 'urate_bin', 'lf']]

count    3139.000000
mean       -0.024435
std         1.930855
min        -4.188213
25%        -1.388213
50%        -0.288213
75%         0.911787
max        13.811787
Name: urate_diff, dtype: float64
lower      43.835616
higher     33.290857
similar    22.873527
Name: urate_bin, dtype: float64


In [9]:
# Creating an empty data frame
temp = pd.DataFrame(columns = ['fips', 'year', 'urate_bin', 'lf'])

for file in files:
    print(file) ########################################
    urate = pd.read_excel('unemployment/' + file, engine = 'openpyxl',
             skiprows = 5, skipfooter = 3,
             names = ['lau_code', 'fips_state', 'fips_county', 'name', 'year', 'na', 'lf',
                      'emp', 'unemp', 'urate'],
             na_values = 'N.A.')

    # Trimming Puerto Rico
    urate = urate[urate['fips_state'] <= 56]
    # Creating FIPS variable
    urate['fips'] = urate.fips_state*1000 + urate['fips_county']

    # Creating label
    urate['urate_nat'] = np.average(urate.unemp/urate.lf)*100
    urate['urate_diff'] = urate.urate - urate.urate_nat
    urate['urate_bin'] = np.where(urate['urate_diff'] < -0.5,
                                  'lower',
                                  np.where(urate['urate_diff'] <= 0.5,
                                           'similar',
                                           'higher'))

    urate = urate[['fips', 'year', 'urate_bin', 'lf']]
    temp = temp.append(urate)

temp = temp.dropna().reset_index(drop = True)
temp = temp.set_index(['fips', 'year'])
temp

laucnty02.xlsx
laucnty03.xlsx
laucnty04.xlsx
laucnty05.xlsx
laucnty06.xlsx
laucnty07.xlsx
laucnty08.xlsx
laucnty09.xlsx
laucnty10.xlsx
laucnty11.xlsx
laucnty12.xlsx
laucnty13.xlsx
laucnty14.xlsx
laucnty15.xlsx
laucnty16.xlsx
laucnty17.xlsx
laucnty18.xlsx


,,urate_bin,lf
fips,year,,
1001,2002,lower,22094
1003,2002,lower,69169
1005,2002,higher,10971
1007,2002,higher,8933
1009,2002,similar,25757
...,...,...,...
56037,2018,similar,21285
56039,2018,lower,15405
56041,2018,similar,9028


In [10]:
temp.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 53367 entries, (1001, 2002) to (56045, 2018)
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   urate_bin  53367 non-null  object
 1   lf         53367 non-null  object
dtypes: object(2)
memory usage: 1.1+ MB


In [11]:
temp['lf'] = temp['lf'].astype(float)

In [12]:
temp.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 53367 entries, (1001, 2002) to (56045, 2018)
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   urate_bin  53367 non-null  object 
 1   lf         53367 non-null  float64
dtypes: float64(1), object(1)
memory usage: 1.1+ MB


In [13]:
df = df.join(temp, how = 'inner')

***
# BDS

[top](#Overview)

In [14]:
bds = pd.read_csv('bds2018_cty.csv', na_values = '(D)')
# print(bds.columns)
# bds

# Trimming
# bds = bds[(bds['year'] >= 2002) & (bds['year'] <= 2018) & (bds.cty < 998)]###############
bds = bds[(bds['year'] >= 2002) & (bds['year'] <= 2018) & (bds.cty < 998)].copy()
# selection = (bds['year'] >= 2002) & (bds['year'] <= 2018) & (bds.cty < 998)
# bds = bds[selection]

# Creating variables we need
bds['fips'] = bds['st']*1000 + bds['cty']
bds['pos_net_jobs'] = (bds.net_job_creation >= 0)*1
bds['emp_estabs'] = bds['emp']/bds.estabs
bds = bds[['fips', 'year', 'pos_net_jobs', 'emp_estabs', 'estabs_entry_rate',
           'estabs_exit_rate']].set_index(['fips', 'year'])

df = df.join(bds, how = 'inner')

***
# CDC
[top](#Overview)

In [15]:
pop = pd.read_table('demographics/population.txt')
pop

,Notes,County,County Code,Yearly July 1st Estimates,Yearly July 1st Estimates Code,Population
0,NaN,"Autauga County, AL",1001.0,2002.0,2002.0,45909.0
1,NaN,"Autauga County, AL",1001.0,2003.0,2003.0,46800.0
2,NaN,"Autauga County, AL",1001.0,2004.0,2004.0,48366.0
3,NaN,"Autauga County, AL",1001.0,2005.0,2005.0,49676.0
4,NaN,"Autauga County, AL",1001.0,2006.0,2006.0,51328.0
...,...,...,...,...,...,...
53508,City are available only for the years prior to...,NaN,NaN,NaN,NaN,NaN
53509,1999 and 2000 due to the addition of population.,NaN,NaN,NaN,NaN,NaN
53510,"16. South Boston City, Virginia (FIPS code 517...",NaN,NaN,NaN,NaN,NaN
53511,"June 30, 1995. This change was made retroactiv...",NaN,NaN,NaN,NaN,NaN


In [16]:
pop = pop.drop(columns = ['Notes', 'County', 'Yearly July 1st Estimates Code']
              ).dropna().rename(columns = {'County Code': 'fips',
                                           'Yearly July 1st Estimates': 'year',
                                          'Population': 'pop'}
                               ).set_index(['fips', 'year'])
df = df.join(pop)

In [17]:
black = pd.read_table('demographics/black.txt')
black = black.drop(columns = ['Notes', 'County', 'Yearly July 1st Estimates Code']
              ).dropna().rename(columns = {'County Code': 'fips',
                                           'Yearly July 1st Estimates': 'year',
                                          'Population': 'black'}
                               ).set_index(['fips', 'year'])
df = df.join(black)

In [18]:
hisp = pd.read_table('demographics/hispanic.txt')
hisp = hisp.drop(columns = ['Notes', 'County', 'Yearly July 1st Estimates Code']
              ).dropna().rename(columns = {'County Code': 'fips',
                                           'Yearly July 1st Estimates': 'year',
                                          'Population': 'hisp'}
                               ).set_index(['fips', 'year'])
df = df.join(hisp)

In [19]:
wrk_age = pd.read_table('demographics/working age pop.txt')
wrk_age = wrk_age.drop(columns = ['Notes', 'County', 'Yearly July 1st Estimates Code']
              ).dropna().rename(columns = {'County Code': 'fips',
                                           'Yearly July 1st Estimates': 'year',
                                          'Population': 'wrk_age'}
                               ).set_index(['fips', 'year'])
df = df.join(wrk_age)

In [20]:
# Calculating percent black population
# Calculating percent hispanic population
# Calculating quasi labor force participation rates

df['pop_pct_black']  = df['black']/df['pop']*100
df['pop_pct_hisp']   = df['hisp']/df['pop']*100
df['lfpr']           = df['lf']/df['wrk_age']*100
df = df.drop(columns = ['lf', 'black', 'hisp', 'wrk_age'])

***
# Social Explorer
[top](#Overview)

In [21]:
land = pd.read_csv('area.csv')
land

,Geo_NAME,Geo_QName,Geo_AREALAND,Geo_AREAWATR,Geo_SUMLEV,Geo_GEOCOMP,Geo_REGION,Geo_DIVISION,Geo_FIPS,Geo_STATE,Geo_COUNTY,SE_T02A_001,SE_T02A_002,SE_T02A_003
0,Autauga County,"Autauga County, Alabama",1539582278,25775735,50,0,3,6,1001,1,1,604.38820,594.43610,9.952067
1,Baldwin County,"Baldwin County, Alabama",4117521611,1133190229,50,0,3,6,1003,1,3,2027.31100,1589.78400,437.527200
2,Barbour County,"Barbour County, Alabama",2291818968,50864716,50,0,3,6,1005,1,5,904.51530,884.87630,19.638980
3,Bibb County,"Bibb County, Alabama",1612480789,9289057,50,0,3,6,1007,1,7,626.16890,622.58230,3.586525
4,Blount County,"Blount County, Alabama",1669961855,15157440,50,0,3,6,1009,1,9,650.62820,644.77590,5.852321
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3216,Vega Baja Municipio,"Vega Baja Municipio, Puerto Rico",118766810,57805843,50,0,9,0,72145,72,145,68.17509,45.85612,22.318960
3217,Vieques Municipio,"Vieques Municipio, Puerto Rico",131488427,552246103,50,0,9,0,72147,72,147,263.99140,50.76797,213.223400
3218,Villalba Municipio,"Villalba Municipio, Puerto Rico",92298550,3622647,50,0,9,0,72149,72,149,37.03539,35.63667,1.398712
3219,Yabucoa Municipio,"Yabucoa Municipio, Puerto Rico",143005172,72593044,50,0,9,0,72151,72,151,83.24294,55.21461,28.028330


In [22]:
land = land[['Geo_FIPS', 'SE_T02A_002']].rename(columns = {'Geo_FIPS': 'fips',
                                                           'SE_T02A_002': 'land'}).set_index('fips')
df = df.join(land)
df['density'] = df['pop']/df['land']
df = df.drop(columns = 'land')

***
# Saving
[Top](#Overview)

We just have a few things to tidy up before saving.

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 52397 entries, (1001, 2002, 'Autauga, AL') to (56045, 2018, 'Weston, WY')
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   pct_d_rgdp         52397 non-null  float64
 1   urate_bin          52397 non-null  object 
 2   pos_net_jobs       52397 non-null  int32  
 3   emp_estabs         52367 non-null  float64
 4   estabs_entry_rate  51477 non-null  float64
 5   estabs_exit_rate   51464 non-null  float64
 6   pop                52397 non-null  float64
 7   pop_pct_black      52260 non-null  float64
 8   pop_pct_hisp       52390 non-null  float64
 9   lfpr               52397 non-null  float64
 10  density            52363 non-null  float64
dtypes: float64(9), int32(1), object(1)
memory usage: 4.6+ MB


In [24]:
# Adding year to the variables while keeping it in the index
df['year'] = df.index.get_level_values('year')
df.sort_index(inplace = True)
df

pct_d_rgdp urate_bin  pos_net_jobs  emp_estabs  \
fips  year GeoName                                                       
1001  2002 Autauga, AL    3.202147     lower             1   12.531208   
      2003 Autauga, AL    1.434404     lower             1   12.598415   
      2004 Autauga, AL   15.061365     lower             1   12.780078   
      2005 Autauga, AL    0.333105    higher             1   12.856784   
      2006 Autauga, AL    7.440034    higher             1   12.832506   
...                            ...       ...           ...         ...   
56045 2014 Weston, WY     2.055429     lower             1    8.410000   
      2015 Weston, WY    12.558802     lower             0    8.415385   
      2016 Weston, WY   -10.381257   similar             0    7.644231   
      2017 Weston, WY    -0.153371     lower             0    7.808081   
      2018 Weston, WY     7.624480     lower             0    8.074468   

                        estabs_entry_rate  estabs_exit_rate      pop  \
fips  year GeoName                                                     
1001  2002 Autauga, AL             11.268             9.256  45909.0   
      2003 Autauga, AL             10.603             9.940  46800.0   
      2004 Autauga, AL             11.140             8.519  48366.0   
      2005 Autauga, AL             11.735             8.673  49676.0   
      2006 Autauga, AL             10.645             8.766  51328.0   
...                                   ...               ...      ...   
56045 2014 Weston, WY               9.694             5.612   7138.0   
      2015 Weston, WY               6.076             8.608   7208.0   
      2016 Weston, WY              13.896             7.444   7220.0   
      2017 Weston, WY               5.941             9.901   6968.0   
      2018 Weston, WY               5.208             9.375   6924.0   

                        pop_pct_black  pop_pct_hisp       lfpr    density  \
fips  year GeoName                                                          
1001  2002 Autauga, AL      17.386569      1.611884  74.841638  77.231178   
      2003 Autauga, AL      17.493590      1.692308  75.093851  78.730077   
      2004 Autauga, AL      17.584667      1.796717  74.459624  81.364507   
      2005 Autauga, AL      17.612127      1.986875  74.920228  83.568276   
      2006 Autauga, AL      17.898613      2.032029  73.641001  86.347380   
...                               ...           ...        ...        ...   
56045 2014 Weston, WY        1.204819      3.950686  87.627044   2.976537   
      2015 Weston, WY        1.054384      3.953940  86.978480   3.005727   
      2016 Weston, WY        1.038781      4.099723  87.816245   3.010731   
      2017 Weston, WY        1.248565      4.118829  87.065369   2.905647   
      2018 Weston, WY        1.299827      3.942808  89.519962   2.887299   

                        year  
fips  year GeoName            
1001  2002 Autauga, AL  2002  
      2003 Autauga, AL  2003  
      2004 Autauga, AL  2004  
      2005 Autauga, AL  2005  
      2006 Autauga, AL  2006  
...                      ...  
56045 2014 Weston, WY   2014  
      2015 Weston, WY   2015  
      2016 Weston, WY   2016  
      2017 Weston, WY   2017  
      2018 Weston, WY   2018  

[52397 rows x 12 columns]

In [25]:
# Checking for NAs
df.isnull().any()

pct_d_rgdp           False
urate_bin            False
pos_net_jobs         False
emp_estabs            True
estabs_entry_rate     True
estabs_exit_rate      True
pop                  False
pop_pct_black         True
pop_pct_hisp          True
lfpr                 False
density               True
year                 False
dtype: bool

In [26]:
print(df.shape)
df.dropna(inplace = True)
df.shape

(52397, 12)


(50834, 12)

In [27]:
# Save the data as a pickle to maintain the hierachical index
df.to_pickle('class_data.pkl')
df.drop(columns = 'year').to_csv('class_data.csv') # need to avoid duplicate year variable in CSV
os.listdir()

['area codebook.txt',
 'area.csv',
 'bds2018_cty.csv',
 'CAGDP9__ALL_AREAS_2001_2019.csv',
 'class_data.csv',
 'class_data.pkl',
 'demographics',
 'unemployment']

Checking to see if it worked!

In [28]:
%reset -f

In [29]:
%whos

Interactive namespace is empty.


In [30]:
import pandas as pd
pd.read_pickle('class_data.pkl')

pct_d_rgdp urate_bin  pos_net_jobs  emp_estabs  \
fips  year GeoName                                                       
1001  2002 Autauga, AL    3.202147     lower             1   12.531208   
      2003 Autauga, AL    1.434404     lower             1   12.598415   
      2004 Autauga, AL   15.061365     lower             1   12.780078   
      2005 Autauga, AL    0.333105    higher             1   12.856784   
      2006 Autauga, AL    7.440034    higher             1   12.832506   
...                            ...       ...           ...         ...   
56045 2014 Weston, WY     2.055429     lower             1    8.410000   
      2015 Weston, WY    12.558802     lower             0    8.415385   
      2016 Weston, WY   -10.381257   similar             0    7.644231   
      2017 Weston, WY    -0.153371     lower             0    7.808081   
      2018 Weston, WY     7.624480     lower             0    8.074468   

                        estabs_entry_rate  estabs_exit_rate      pop  \
fips  year GeoName                                                     
1001  2002 Autauga, AL             11.268             9.256  45909.0   
      2003 Autauga, AL             10.603             9.940  46800.0   
      2004 Autauga, AL             11.140             8.519  48366.0   
      2005 Autauga, AL             11.735             8.673  49676.0   
      2006 Autauga, AL             10.645             8.766  51328.0   
...                                   ...               ...      ...   
56045 2014 Weston, WY               9.694             5.612   7138.0   
      2015 Weston, WY               6.076             8.608   7208.0   
      2016 Weston, WY              13.896             7.444   7220.0   
      2017 Weston, WY               5.941             9.901   6968.0   
      2018 Weston, WY               5.208             9.375   6924.0   

                        pop_pct_black  pop_pct_hisp       lfpr    density  \
fips  year GeoName                                                          
1001  2002 Autauga, AL      17.386569      1.611884  74.841638  77.231178   
      2003 Autauga, AL      17.493590      1.692308  75.093851  78.730077   
      2004 Autauga, AL      17.584667      1.796717  74.459624  81.364507   
      2005 Autauga, AL      17.612127      1.986875  74.920228  83.568276   
      2006 Autauga, AL      17.898613      2.032029  73.641001  86.347380   
...                               ...           ...        ...        ...   
56045 2014 Weston, WY        1.204819      3.950686  87.627044   2.976537   
      2015 Weston, WY        1.054384      3.953940  86.978480   3.005727   
      2016 Weston, WY        1.038781      4.099723  87.816245   3.010731   
      2017 Weston, WY        1.248565      4.118829  87.065369   2.905647   
      2018 Weston, WY        1.299827      3.942808  89.519962   2.887299   

                        year  
fips  year GeoName            
1001  2002 Autauga, AL  2002  
      2003 Autauga, AL  2003  
      2004 Autauga, AL  2004  
      2005 Autauga, AL  2005  
      2006 Autauga, AL  2006  
...                      ...  
56045 2014 Weston, WY   2014  
      2015 Weston, WY   2015  
      2016 Weston, WY   2016  
      2017 Weston, WY   2017  
      2018 Weston, WY   2018  

[50834 rows x 12 columns]